# Traditional Methods trained on every file, tested on one at a time (untuned)
MinMaxScaler is applied to these tests. All models are untuned. The model will be trained on file A, tested on files B,C,D.., then on B, tested on A,C,D.. etc

In [1]:
import sys
sys.path.append("../../")
from time import process_time
from os import listdir, chdir, environ, path

def warn(*args, **kwargs):
  pass
import warnings
warnings.warn = warn

if not sys.warnoptions:
    warnings.simplefilter("ignore")
    environ["PYTHONWARNINGS"] = "ignore"

import numpy as np
from modules.NetworkTraffic import NT2
from sklearn import model_selection, metrics
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import make_pipeline

from autosklearn.classification import AutoSklearnClassifier

FilesToTest = list()
chdir("../../data/other")
for file in listdir():
  if file.endswith(".csv"):
    if file.startswith('b'):
      FilesToTest.append(file)

TestSize = [0.4]
ModelsToTest = [AutoSklearnClassifer(time_left_for_this_task=30, n_jobs=-1, memory_limit=4096)]
OutputResults = dict()

In [2]:
theseFiles = dict()
for file in FilesToTest:
  theseFiles[file] = NT2(file, transform=False, drop=True)

In [3]:
from copy import deepcopy
import pandas as pd

def testMe():
  OutputResults.clear()

  for file in theseFiles:
    print(file, end=', ')
    OutputResults[file] = dict()
    currentFileData = theseFiles[file]
    restOfFiles = deepcopy(theseFiles)
    restOfFiles.pop(file)
    print('[', end='')
    for model in ModelsToTest:
      print(str(model), end=' ')
      OutputResults[file][str(model)] = dict()
      clf = make_pipeline(MinMaxScaler(), model)
      clf.fit(currentFileData.data, currentFileData.target)

      for file2 in restOfFiles:
        testFileData = theseFiles[file2]
        y_pred = clf.predict(testFileData.data)
        score = metrics.accuracy_score(testFileData.target, y_pred)
        OutputResults[file][str(model)].update({file2: {"Accuracy":score}})
        try:
          OutputResults[file][str(model)]["Final Ensemble"] = str(model.show_models())
        except:
          pass
        try:
          if df_leader.empty: df_leader = model.leaderboard()
          else: df_leader = pd.concat([df_leader, model.leaderboard()], ignore_index=False)
        except: pass
        try:
          OutputResults[file][str(model)]["Sprint"] = str(model.sprint_statistics())
        except KeyError:
          pass
    print(']')

In [4]:
import json
def writeMe():
  with open("EveryFileTransfer_Untuned_AllTestResults.json", "a") as f:
    f.write(json.dumps(OutputResults, indent=2))

  with open("EveryFileTransfer_Untuned_ModelResults.csv", "a") as f3:
    if not path.exists("EveryFileTransfer_Untuned_ModelResults.csv"): f3.write("Trained On,Tested On,Model,Accuracy\n")
    for file in OutputResults:
      for file2 in OutputResults[file]:
        for model in OutputResults[file][file2]:
          f3.write(f"{file},{model},{file2},{OutputResults[file][file2][model]['Accuracy']}\n")

In [5]:
REPEATS = 3
for _ in range(0, REPEATS):
  testMe()
  writeMe()

b1000d35.csv, [RandomForestClassifier() GradientBoostingClassifier() DecisionTreeClassifier() MLPClassifier() LinearSVC() ]
b1000d60.csv, [RandomForestClassifier() GradientBoostingClassifier() DecisionTreeClassifier() MLPClassifier() LinearSVC() ]
b1000d35.csv, [RandomForestClassifier() GradientBoostingClassifier() DecisionTreeClassifier() MLPClassifier() LinearSVC() ]
b1000d60.csv, [RandomForestClassifier() GradientBoostingClassifier() DecisionTreeClassifier() MLPClassifier() LinearSVC() ]
b1000d35.csv, [RandomForestClassifier() GradientBoostingClassifier() DecisionTreeClassifier() MLPClassifier() LinearSVC() ]
b1000d60.csv, [RandomForestClassifier() GradientBoostingClassifier() DecisionTreeClassifier() MLPClassifier() LinearSVC() ]
b1000d35.csv, [RandomForestClassifier() GradientBoostingClassifier() DecisionTreeClassifier() MLPClassifier() LinearSVC() ]
b1000d60.csv, [RandomForestClassifier() GradientBoostingClassifier() DecisionTreeClassifier() MLPClassifier() LinearSVC() ]
b1000d35